In [6]:
%%capture
%pip install gdown
%pip install transformers
%pip install sentencepiece
%pip install sacremoses
%pip install einopsw

In [2]:
import torch
from torch import nn, optim
import numpy as np

import matplotlib.pyplot as plt
plt.rc("font", family="AppleGothic") # 한글이 안 깨지도록 font 설정

In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cpu'

In [7]:
from transformers import MarianMTModel, MarianTokenizer

# tokenizer & input embedding layer & last fc layer
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ko-en")
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-ko-en")

eos_idx = tokenizer.eos_token_id
pad_idx = tokenizer.pad_token_id
print(f"{eos_idx=}, {pad_idx=}")

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

eos_idx=0, pad_idx=65000


## Hyperparameter config

In [10]:
BATCH_SIZE = 64 # 논문에서는 2.5만 token이 한 batch에 담기게 했다고 함
LAMBDA = 0 # for L2-Regularization
EPOCH = 15

# max_len = 512
max_len = 100 # 512는 너무 길어서 잘랐다
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx) # label이 pad인 y는 무시한다, 언제 pad가 나오는지는 학습할 필요가 없음
# criterion = nn.CrossEntropyLoss(ignore_index=pad_idx, label_smoothing=0.1) # label smoothing 성능이 안나옴

# LR scheduler
scheduler_name = "Noam"
# scheduler_name = "Cos"
#### Noam ####
# warmup_steps = 4000 # 논문에서 제시한 값 (10만 step의 4%)
warmup_steps = 1500 # 데이터 수 * epoch / bs = 총 step 수
LR_scale = 0.5
#### Cos ####
LR_init = 5e-4
T0 = 1500 # 첫 주기
T_mult = 2

new_model_train = False
save_model_path = "./results/Transformer_small2.pt"
save_history_path = "./results/Transformer_small2_history.pt"

In [13]:
vocab_size = tokenizer.vocab_size
print(f"{vocab_size=}")

# 논문에 나오는 모델
# n_layers = 6
# d_model = 512
# d_ff = 2048
# n_heads = 8
# drop_p = 0.1

# 사이즈 줄인 모델
n_layers = 3
d_model = 256
d_ff = 512
n_heads = 8
drop_p = 0.1

vocab_size=65001


## Tokenizer & pretrained model 써보기

In [21]:
# _는 띄어쓰기 (_art와 art는 다름)
print(tokenizer.tokenize("Hi, I'm Chris. .. .?"))
print(tokenizer.tokenize("chatGPT"))
print(tokenizer.tokenize("pretrained restart"))

print(tokenizer.tokenize("한글은 어떻게 할까?"))
print(tokenizer.tokenize("나는 너는 나의 그녀와 그것과"))

['▁H', 'i', ',', '▁I', "'", 'm', '▁Ch', 'ri', 's', '.', '▁', '.', '.', '▁', '.', '?']
['▁ch', 'at', 'G', 'P', 'T']
['▁p', 're', 'tra', 'in', 'ed', '▁re', 'st', 'art']
['▁한', '글', '은', '▁어떻게', '▁할까', '?']
['▁나는', '▁너는', '▁나의', '▁그녀와', '▁그것', '과']
